<a href="https://colab.research.google.com/github/GeulNoon/GeulNoon/blob/Start/Kobart%2C%20TextRank%20%EB%B9%84%EA%B5%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentence Transformers 학습

In [1]:
pip install sentence-transformers datasets

     |████████████████████████████████| 78 kB 2.6 MB/s 
     |████████████████████████████████| 298 kB 33.5 MB/s 
     |████████████████████████████████| 3.1 MB 33.4 MB/s 
     |████████████████████████████████| 3.3 MB 37.3 MB/s 
     |████████████████████████████████| 1.2 MB 37.9 MB/s 
     |████████████████████████████████| 61 kB 523 kB/s 
     |████████████████████████████████| 596 kB 45.3 MB/s 
     |████████████████████████████████| 895 kB 52.0 MB/s 
     |████████████████████████████████| 243 kB 47.5 MB/s 
     |████████████████████████████████| 1.1 MB 36.7 MB/s 
     |████████████████████████████████| 132 kB 48.1 MB/s 
     |████████████████████████████████| 192 kB 49.9 MB/s 
     |████████████████████████████████| 160 kB 48.3 MB/s 
     |████████████████████████████████| 271 kB 51.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=d754a37b93254d635b269216d3dc5d728f367cebb0949d5972b3a72188958b16
  Stored in

In [2]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [3]:
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [4]:
model_name = "klue/roberta-base"

In [5]:
train_batch_size = 32
num_epochs = 4
model_save_path = "output/training_klue_sts_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [6]:
embedding_model = models.Transformer(model_name)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [7]:
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

In [8]:
model = SentenceTransformer(modules=[embedding_model, pooler])

2021-12-08 16:19:35 - Use pytorch device: cuda


In [9]:
datasets = load_dataset("klue", "sts")

Downloading:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 11668
    })
    validation: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 519
    })
})

In [11]:
datasets["train"][0]

{'guid': 'klue-sts-v1_train_00000',
 'labels': {'binary-label': 1, 'label': 3.7, 'real-label': 3.714285714285714},
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'source': 'airbnb-rtt'}

In [12]:
testsets = load_dataset("kor_nlu", "sts")

Downloading:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/89.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/66.1k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset kor_nlu downloaded and prepared to /root/.cache/huggingface/datasets/kor_nlu/sts/1.0.0/4facbba77df60b0658056ced2052633e681a50187b9428bd5752ebd59d332ba8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
testsets

DatasetDict({
    train: Dataset({
        features: ['genre', 'filename', 'year', 'id', 'score', 'sentence1', 'sentence2'],
        num_rows: 5703
    })
    validation: Dataset({
        features: ['genre', 'filename', 'year', 'id', 'score', 'sentence1', 'sentence2'],
        num_rows: 1471
    })
    test: Dataset({
        features: ['genre', 'filename', 'year', 'id', 'score', 'sentence1', 'sentence2'],
        num_rows: 1379
    })
})

In [14]:
testsets["test"][0]

{'filename': 2,
 'genre': 1,
 'id': 24,
 'score': 2.5,
 'sentence1': '한 소녀가 머리를 스타일링하고 있다.',
 'sentence2': '한 소녀가 머리를 빗고 있다.',
 'year': 6}

In [15]:
train_samples = []
dev_samples = []
test_samples = []

# KLUE STS 내 훈련, 검증 데이터 예제 변환
for phase in ["train", "validation"]:
    examples = datasets[phase]

    for example in examples:
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화

        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]], 
            label=score,
        )

        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

# KorSTS 내 테스트 데이터 예제 변환
for example in testsets["test"]:
    score = float(example["score"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples.append(inp_example)

In [16]:
train_samples[0].texts, train_samples[0].label

(['숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
  '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.'],
 0.74)

In [17]:
test_samples[0].texts, test_samples[0].label

(['한 소녀가 머리를 스타일링하고 있다.', '한 소녀가 머리를 빗고 있다.'], 0.5)

In [18]:
train_dataloader = DataLoader(
    train_samples,
    shuffle=True,
    batch_size=train_batch_size,
)
train_loss = losses.CosineSimilarityLoss(model=model)

In [19]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_samples,
    name="sts-dev",
)

In [20]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data for warm-up
logging.info(f"Warmup-steps: {warmup_steps}")

2021-12-08 16:19:45 - Warmup-steps: 146


In [21]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-12-08 16:26:12 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2021-12-08 16:26:16 - Cosine-Similarity :	Pearson: 0.8809	Spearman: 0.8763
2021-12-08 16:26:16 - Manhattan-Distance:	Pearson: 0.8784	Spearman: 0.8721
2021-12-08 16:26:16 - Euclidean-Distance:	Pearson: 0.8784	Spearman: 0.8720
2021-12-08 16:26:16 - Dot-Product-Similarity:	Pearson: 0.8699	Spearman: 0.8640
2021-12-08 16:26:16 - Save model to output/training_klue_sts_klue-roberta-base-2021-12-08_16-19-15


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-12-08 16:32:33 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 1:
2021-12-08 16:32:37 - Cosine-Similarity :	Pearson: 0.8885	Spearman: 0.8871
2021-12-08 16:32:37 - Manhattan-Distance:	Pearson: 0.8896	Spearman: 0.8842
2021-12-08 16:32:37 - Euclidean-Distance:	Pearson: 0.8894	Spearman: 0.8840
2021-12-08 16:32:37 - Dot-Product-Similarity:	Pearson: 0.8779	Spearman: 0.8726
2021-12-08 16:32:37 - Save model to output/training_klue_sts_klue-roberta-base-2021-12-08_16-19-15


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-12-08 16:38:54 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 2:
2021-12-08 16:38:58 - Cosine-Similarity :	Pearson: 0.8918	Spearman: 0.8905
2021-12-08 16:38:58 - Manhattan-Distance:	Pearson: 0.8920	Spearman: 0.8868
2021-12-08 16:38:58 - Euclidean-Distance:	Pearson: 0.8922	Spearman: 0.8869
2021-12-08 16:38:58 - Dot-Product-Similarity:	Pearson: 0.8792	Spearman: 0.8736
2021-12-08 16:38:58 - Save model to output/training_klue_sts_klue-roberta-base-2021-12-08_16-19-15


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-12-08 16:45:14 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 3:
2021-12-08 16:45:18 - Cosine-Similarity :	Pearson: 0.8917	Spearman: 0.8911
2021-12-08 16:45:18 - Manhattan-Distance:	Pearson: 0.8917	Spearman: 0.8874
2021-12-08 16:45:18 - Euclidean-Distance:	Pearson: 0.8921	Spearman: 0.8877
2021-12-08 16:45:18 - Dot-Product-Similarity:	Pearson: 0.8798	Spearman: 0.8752
2021-12-08 16:45:18 - Save model to output/training_klue_sts_klue-roberta-base-2021-12-08_16-19-15


In [22]:
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')

2021-12-08 16:45:20 - Load pretrained SentenceTransformer: output/training_klue_sts_klue-roberta-base-2021-12-08_16-19-15
2021-12-08 16:45:21 - Use pytorch device: cuda


In [23]:
test_evaluator(model, output_path=model_save_path)

2021-12-08 16:45:21 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2021-12-08 16:45:31 - Cosine-Similarity :	Pearson: 0.7656	Spearman: 0.7615
2021-12-08 16:45:31 - Manhattan-Distance:	Pearson: 0.7645	Spearman: 0.7631
2021-12-08 16:45:31 - Euclidean-Distance:	Pearson: 0.7630	Spearman: 0.7622
2021-12-08 16:45:31 - Dot-Product-Similarity:	Pearson: 0.7381	Spearman: 0.7317


0.7630815879938855

## Kobart 요약모델과 TextRank 알고리즘 요약 성능 비교

In [24]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from pandas import DataFrame

In [27]:
data = pd.read_csv('./test.csv', encoding='CP949')
data

,직접 요약,Kobart,TextRank
0,최근 들어 논의되고 있는 창조 도시는 인재들을 위한 문화 및 거주 환경의 창조성이 ...,최근 들어 도시의 경쟁력 향상을 위한 새로운 전략의 하나로 창조 도시에 대한 논의가...,창조 도시는 창조적 인재들이 창의성을 발휘할 수 있는 환경을 갖춘 도시이다.
1,창조 도시의 주된 동력을 창조 산업으로 보는 관점에서는 창조 산업이 도시에 인적·사...,"창조 산업을 중시하는 관점에서는, 창조 산업이 도시에 인적·사회적·문화적·경제적 다...",창조 도시의 주된 동력을 창조 산업으로 볼 것인가 창조 계층으로 볼 것인가에 대해서...
2,창조 계층을 중시하는 관점에서는 창조 계층이 모여 인재 네트워크인 창조 자본을 형성...,개인의 창의력으로 부가가치를 창출하는 창조 계층이 모여서 인재 네트워크인 창조 자본...,창조 계층을 끌어들이고 유지하는 것이 도시의 경쟁력을 제고하는 관건이 된다.
3,"한 도시가 창조 도시로 성장하려면 창조 환경이 먼저 마련되어야 하며, 랜드리는 창조...",한 도시가 창조 도시로 성장하려면 창조 산업과 창조 계층을 유인하는 창조 환경이 먼...,"창조성의 근본 동력을 무엇으로 보든, 한 도시가 창조 도시로 성장하려면 창조 산업과..."
4,창조 도시를 만드는 과정에서 위험이 수반되기도 하기 때문에 도시 고유의 특성을 면밀...,하루아침에 인위적으로 만들어지지 않으며 추진 과정에서 위험이 수반되기도 하는 창조 ...,"창조 도시를 만들기 위해서는 도시 고유의 특성을 면밀히 고찰하여 창조 산업, 창조 ..."
5,"현재 전국적으로 1,000여 개 이상의 지역 축제가 열리고 있지만, 주로 대중 공연...","1990년대에 지방 자치제가 본격적으로 시행된 이후 현재 전국적으로 1,000여 개...",지역 축제는 지역 주민의 단합에 이바지하고 지역 경제를 활성화시키는 역할을 한다. ...
6,지역 축제를 내실화할 수 있는 첫째 방안은 그 지역만의 개성을 담는 것으로 전통문화...,지역 축제를 내실화할 수 있는 첫째 방안은 그 지역만의 개성을 담아 다른 지역과 차...,성공적이라고 평가받고 있는 지역 축제들의 사례를 살펴보자. 그러므로 성공적인 지역...
7,내실화의 둘째 방안은 지역 축제의 효율적 운영으로 주제가 유사한 축제를 통합하고 예...,지역 축제를 차별화할 수 있는 좋은 아이디어가 있어도 예산이 부족하면 내실 있게 운...,내실화의 둘째 방안은 지역 축제의 효율적 운영이다. 지역만의 개성을 담아 지역 축제...
8,지역 축제를 내실화하기 위해서는 내용 측면에서 지역의 개성을 담아 다른 지역 축제와...,내용 측면에서 지역의 개성을 담아 다른 지역 축제와 차별화하는 것이 중요하며 운영 ...,지역 축제를 내실화하기 위해서는 무엇보다 내용 측면에서 지역의 개성을 담아 다른 지...
9,"채권은 사업에 필요한 자금을 조달하기 위해 발행하는 유가증권으로, 국채나 회사채 등...","채권은 사업에 필요한 자금을 조달하기 위해 발행하는 유가증권으로, 국채나 회사채 등...","채권의 액면 금액, 액면 이자율, 만기일 등의 지급 조건은 채권 발행 시 정해지며,..."


In [28]:
df = pd.DataFrame(data)
pd.options.display.float_format = '{:.6f}'.format

In [29]:
c1=[]
c2=[]

for i in range (len(df.index)):
  a = data.iloc[i]

  embedding1 = model.encode(a[0], convert_to_tensor=True)
  embedding2 = model.encode(a[1], convert_to_tensor=True)
  embedding3 = model.encode(a[2], convert_to_tensor=True)

  cosine_score1 = util.pytorch_cos_sim(embedding1, embedding2)
  c1.append( float(cosine_score1[0][0]))
  cosine_score2 = util.pytorch_cos_sim(embedding1, embedding3)
  c2.append(float(cosine_score2[0][0]))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
df['직접 요약과 KoBART 요약 유사도'] = c1
df['직접 요약과 TextRank 요약 유사도'] = c2

In [31]:
df

,직접 요약,Kobart,TextRank,직접 요약과 KoBART 요약 유사도,직접 요약과 TextRank 요약 유사도
0,최근 들어 논의되고 있는 창조 도시는 인재들을 위한 문화 및 거주 환경의 창조성이 ...,최근 들어 도시의 경쟁력 향상을 위한 새로운 전략의 하나로 창조 도시에 대한 논의가...,창조 도시는 창조적 인재들이 창의성을 발휘할 수 있는 환경을 갖춘 도시이다.,0.882332,0.779450
1,창조 도시의 주된 동력을 창조 산업으로 보는 관점에서는 창조 산업이 도시에 인적·사...,"창조 산업을 중시하는 관점에서는, 창조 산업이 도시에 인적·사회적·문화적·경제적 다...",창조 도시의 주된 동력을 창조 산업으로 볼 것인가 창조 계층으로 볼 것인가에 대해서...,0.895760,0.558853
2,창조 계층을 중시하는 관점에서는 창조 계층이 모여 인재 네트워크인 창조 자본을 형성...,개인의 창의력으로 부가가치를 창출하는 창조 계층이 모여서 인재 네트워크인 창조 자본...,창조 계층을 끌어들이고 유지하는 것이 도시의 경쟁력을 제고하는 관건이 된다.,0.886613,0.642029
3,"한 도시가 창조 도시로 성장하려면 창조 환경이 먼저 마련되어야 하며, 랜드리는 창조...",한 도시가 창조 도시로 성장하려면 창조 산업과 창조 계층을 유인하는 창조 환경이 먼...,"창조성의 근본 동력을 무엇으로 보든, 한 도시가 창조 도시로 성장하려면 창조 산업과...",0.834761,0.691118
4,창조 도시를 만드는 과정에서 위험이 수반되기도 하기 때문에 도시 고유의 특성을 면밀...,하루아침에 인위적으로 만들어지지 않으며 추진 과정에서 위험이 수반되기도 하는 창조 ...,"창조 도시를 만들기 위해서는 도시 고유의 특성을 면밀히 고찰하여 창조 산업, 창조 ...",0.903932,0.843956
5,"현재 전국적으로 1,000여 개 이상의 지역 축제가 열리고 있지만, 주로 대중 공연...","1990년대에 지방 자치제가 본격적으로 시행된 이후 현재 전국적으로 1,000여 개...",지역 축제는 지역 주민의 단합에 이바지하고 지역 경제를 활성화시키는 역할을 한다. ...,0.860984,0.425990
6,지역 축제를 내실화할 수 있는 첫째 방안은 그 지역만의 개성을 담는 것으로 전통문화...,지역 축제를 내실화할 수 있는 첫째 방안은 그 지역만의 개성을 담아 다른 지역과 차...,성공적이라고 평가받고 있는 지역 축제들의 사례를 살펴보자. 그러므로 성공적인 지역...,0.913223,0.416810
7,내실화의 둘째 방안은 지역 축제의 효율적 운영으로 주제가 유사한 축제를 통합하고 예...,지역 축제를 차별화할 수 있는 좋은 아이디어가 있어도 예산이 부족하면 내실 있게 운...,내실화의 둘째 방안은 지역 축제의 효율적 운영이다. 지역만의 개성을 담아 지역 축제...,0.687193,0.648467
8,지역 축제를 내실화하기 위해서는 내용 측면에서 지역의 개성을 담아 다른 지역 축제와...,내용 측면에서 지역의 개성을 담아 다른 지역 축제와 차별화하는 것이 중요하며 운영 ...,지역 축제를 내실화하기 위해서는 무엇보다 내용 측면에서 지역의 개성을 담아 다른 지...,0.947958,0.796810
9,"채권은 사업에 필요한 자금을 조달하기 위해 발행하는 유가증권으로, 국채나 회사채 등...","채권은 사업에 필요한 자금을 조달하기 위해 발행하는 유가증권으로, 국채나 회사채 등...","채권의 액면 금액, 액면 이자율, 만기일 등의 지급 조건은 채권 발행 시 정해지며,...",0.922939,0.778584


In [32]:
mean_df = df['직접 요약과 KoBART 요약 유사도'].mean()
mean_df2 = df['직접 요약과 TextRank 요약 유사도'].mean()
print("직접 요약과 KoBART 요약 평균 유사도: " + str(mean_df))
print("직접 요약과 TextRank 요약 평균 유사도: " + str(mean_df2))

직접 요약과 KoBART 요약 평균 유사도: 0.8821756978829701
직접 요약과 TextRank 요약 평균 유사도: 0.6732033083836237
